# 1. Import all the required modules

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# 2. Load the datasets

In [2]:
train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                            train=False,
                            transform=transforms.ToTensor())

# 3. Set parameters

In [3]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters/(len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

# 4. Make the datasets iterable 

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

# 5. Define Long short term memory model 

In [5]:
class LSTMModel(nn.Module):
    def __init__(self,input_dim,hidden_dim,layer_dim,output_dim):
        super(LSTMModel,self).__init__()
    
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
        self.lstm = nn.LSTM(input_dim,hidden_dim,layer_dim,batch_first=True)
        self.fc = nn.Linear(hidden_dim,output_dim)
        
    def forward(self,x):
        if torch.cuda.is_available():
            hO = Variable(torch.zeros(self.layer_dim,x.size(0),self.hidden_dim).cuda())
            cO = Variable(torch.zeros(self.layer_dim,x.size(0),self.hidden_dim).cuda())
        else:
            hO = Variable(torch.zeros(self.layer_dim,x.size(0),self.hidden_dim))
            cO = Variable(torch.zeros(self.layer_dim,x.size(0),self.hidden_dim))
            
        out, (hn,cn) = self.lstm(x,(hO,cO))
        out = self.fc(out[:,-1,:])
        
        return out

# 6. Set parameters for model 

In [6]:
input_dim = 28
hidden_dim = 100
layer_dim = 2
output_dim = 10
learning_rate = 0.1

# 7. Instantiate model, criterion and optimizer 

In [7]:
model = LSTMModel(input_dim,hidden_dim,layer_dim,output_dim)
if torch.cuda.is_available():
    model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 8. Train the model

In [8]:
seq_dim = 28
iter = 0
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.view(-1,seq_dim,input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1,seq_dim,input_dim))
            labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        iter += 1
        if iter % 500 == 0:
            correct = 0
            total = 0
            for images,labels in test_loader:
                if torch.cuda.is_available():
                    images = Variable(images.view(-1,seq_dim,input_dim).cuda())
                else:
                    images = Variable(images.view(-1,seq_dim,input_dim))
                
                outputs = model(images)
                _,predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted.cpu() == labels.cpu()).sum()
                
            accuracy = 100* correct/total
            
            print('Iteration:{} | Loss:{} | Accuracy:{}'.format(iter,loss.data[0],accuracy))

/home/shubham/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iteration:500 | Loss:2.30165696144104 | Accuracy:11
Iteration:1000 | Loss:2.007586717605591 | Accuracy:25
Iteration:1500 | Loss:0.7109007239341736 | Accuracy:77
Iteration:2000 | Loss:0.5312872529029846 | Accuracy:88
Iteration:2500 | Loss:0.28135937452316284 | Accuracy:93
Iteration:3000 | Loss:0.1154506504535675 | Accuracy:95


# Save trained model

In [9]:
torch.save(model.state_dict(),'LSTM-Model.pkl')